# Submission

In [689]:
# Magic to automatically update imports if functions in utils are changed
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [690]:
submission_with_whole_train = False

# Feature engineer (option 1)

In [691]:
from sklearn.model_selection import train_test_split

stores_train = pd.read_csv("data/stores_train.csv")
stores_test = pd.read_csv("data/stores_test.csv")
stores_extra = pd.read_csv("data/stores_extra.csv")
if not submission_with_whole_train:
    stores_train, stores_val = train_test_split(stores_train, test_size=0.2, random_state=0)

In [692]:
from utils import split_plaace_cat


store_dataframes = {
    "train": stores_train, 
    "extra": stores_extra, 
    "test": stores_test, 
    }

if not submission_with_whole_train:
    store_dataframes["val"] = stores_val

for df_name, df in store_dataframes.items():
    df = split_plaace_cat(df)

In [693]:
from remove_outlier import remove_outliers
store_dataframes["train"] = remove_outliers(store_dataframes["train"], "plaace_cat_4")

from outlier_utils import remove_low_revenue
store_dataframes["train"] = remove_low_revenue(store_dataframes["train"], low_rev_limit=0.05)

In [694]:
store_dataframes["train"].shape

(9856, 16)

In [695]:
store_dataframes["train"]["log_revenue"] = store_dataframes["train"].revenue.apply(lambda x: np.log1p(x))
if not submission_with_whole_train:
    store_dataframes["val"]["log_revenue"] = store_dataframes["val"].revenue.apply(lambda x: np.log1p(x))

In [762]:
from utils import mean_rev_of_competitor, log_mean_rev_of_competitor, create_geographical_columns, create_chain_and_mall_columns, generate_chain_rev_dict, generate_plaace_rev_dict, create_mean_chain_rev_col

chain_count = stores_train["chain_name"].value_counts().to_dict()

for df_name, df in tqdm(store_dataframes.items()):
    store_dataframes[df_name] = create_geographical_columns(df)
    store_dataframes[df_name] = create_chain_and_mall_columns(df, chain_count, lower_limit=1)

chain_rev_dict, log_bounded_chain_rev_dict = generate_chain_rev_dict(store_dataframes["train"], quantile=0)

for i in tqdm(range(1, 5)):
    rev_plaace_dict, mean_plaace_revenue, log_rev_plaace_dict, log_mean_plaace_revenue = generate_plaace_rev_dict(store_dataframes["train"], i, quantile=0)
    for df_name, df in store_dataframes.items():
            store_dataframes[df_name] = mean_rev_of_competitor(store_dataframes[df_name], i, rev_dict=rev_plaace_dict, mean_revenue=mean_plaace_revenue)
            store_dataframes[df_name] = log_mean_rev_of_competitor(store_dataframes[df_name], i, log_rev_dict=log_rev_plaace_dict, log_mean_revenue=log_mean_plaace_revenue)
    
for df_name, df in tqdm(store_dataframes.items()):
    store_dataframes[df_name] = create_mean_chain_rev_col(df, bounded_chain_revs=chain_rev_dict, log_bounded_chain_revs=log_bounded_chain_rev_dict)

100%|██████████| 4/4 [00:00<00:00, 190.35it/s]


In [698]:
from utils import concat_df_keep_unq_index
concat_df = concat_df_keep_unq_index(stores_train, stores_extra)

!NB next cell takes 3 minutes to run (if comp_plaace_gran. = [1, 2, 3, 4]) (on M1 Mac with 16GB RAM)

(outdated) If you have the .csv files temp_data/closest_comp_\{df_name\}, skip running this cell and run the cell below it instead

In [699]:
from utils import find_dist_to_nearest_comp

nearest_comp_plaace_cat_gran = [1, 2, 3, 4]
n_nearest_comp = [1, 2, 3, 4, 5, 7, 10]

store_dataframes["train"] = find_dist_to_nearest_comp(
    store_dataframes["train"], 
    nearest_comp_plaace_cat_gran, 
    n_nearest_comp, 
    training=True, 
    training_df=concat_df,
    )

In [700]:
if not submission_with_whole_train:
    store_dataframes["val"] = find_dist_to_nearest_comp(
        store_dataframes["val"], 
        nearest_comp_plaace_cat_gran, 
        n_nearest_comp, 
        training=True, 
        training_df=concat_df,
        )

In [701]:
store_dataframes["test"] = find_dist_to_nearest_comp(
    store_dataframes["test"], 
    nearest_comp_plaace_cat_gran, 
    n_nearest_comp, 
    training=False,
    training_df=concat_df
)

In [702]:
comp_plaace_cols = list(store_dataframes["train"].columns[-56:])

In [703]:
dist_dict = store_dataframes["train"][['log_revenue'] + comp_plaace_cols].corr().iloc[0].to_dict()
sorted_relevant_dist_cols = [[k, v] for k, v in sorted(dist_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
sorted_relevant_dist_cols

[['log_revenue', 1.0],
 ['sum_dist_to_nearest_10_comp_plaace_1', -0.11791123528859537],
 ['mean_dist_to_nearest_10_comp_plaace_1', -0.117911235288595],
 ['mean_dist_to_nearest_7_comp_plaace_1', -0.11431968464214541],
 ['sum_dist_to_nearest_7_comp_plaace_1', -0.1143196846421449],
 ['mean_dist_to_nearest_5_comp_plaace_1', -0.11000682292553704],
 ['sum_dist_to_nearest_5_comp_plaace_1', -0.11000682292553673],
 ['sum_dist_to_nearest_4_comp_plaace_1', -0.10649073608996629],
 ['mean_dist_to_nearest_4_comp_plaace_1', -0.10649073608996629],
 ['mean_dist_to_nearest_3_comp_plaace_1', -0.09991523665621461],
 ['sum_dist_to_nearest_3_comp_plaace_1', -0.09991523665621417],
 ['sum_dist_to_nearest_2_comp_plaace_1', -0.09684610738958518],
 ['mean_dist_to_nearest_2_comp_plaace_1', -0.09684610738958518],
 ['sum_dist_to_nearest_1_comp_plaace_1', -0.08361246404990565],
 ['mean_dist_to_nearest_1_comp_plaace_1', -0.08361246404990565],
 ['sum_dist_to_nearest_1_comp_plaace_2', -0.055627306424984625],
 ['mean_di

In [704]:
comp_relevant_cols = sorted_relevant_dist_cols[1:14:2]
comp_relevant_cols = [r[0] for r in comp_relevant_cols]

In [705]:
new_comp_relevant_cols = [
    'sum_dist_to_nearest_10_comp_plaace_1',
    'sum_dist_to_nearest_3_comp_plaace_1',
    'sum_dist_to_nearest_1_comp_plaace_1', 
    'sum_dist_to_nearest_3_comp_plaace_2',
    'sum_dist_to_nearest_1_comp_plaace_2',
    'sum_dist_to_nearest_1_comp_plaace_3',
    ]

!NB next cell takes 15 minutes to run. Grab a coffee or somtething while you wait :) (on M1 Mac with 16GB RAM)

If you have the .csv files temp_data/closest_bus_stop_\{df_name\}, skip running this cell and run the cell below it instead

In [706]:
from bus_utils import find_closest_bus_stop

bus_stop_n = [1, 2, 3, 5, 7, 10, 15 ,25, 50, 100]
bus_mean = True
bus_sum = True

bus_stop_columns = []

if(bus_sum):
    bus_stop_columns += [f"closest_bus_stop_sum_{i}" for i in bus_stop_n]

if(bus_mean):
    bus_stop_columns += [f"closest_bus_stop_mean_{i}" for i in bus_stop_n]


for df_name, df in tqdm(store_dataframes.items()):
    store_dataframes[df_name] = find_closest_bus_stop(df, bus_stop_n, _sum=bus_sum, _mean=bus_mean)    

100%|██████████| 4/4 [12:11<00:00, 182.82s/it]


In [707]:
bus_dict = store_dataframes["train"][['log_revenue'] + bus_stop_columns].corr().iloc[0].to_dict()
bus_sorted_relevant_dist_cols = [[k, v] for k, v in sorted(bus_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
bus_sorted_relevant_dist_cols

[['log_revenue', 1.0],
 ['closest_bus_stop_mean_15', -0.047443014968288434],
 ['closest_bus_stop_sum_15', -0.047443014968288115],
 ['closest_bus_stop_mean_10', -0.04495627097222023],
 ['closest_bus_stop_sum_10', -0.044956270972220005],
 ['closest_bus_stop_sum_25', -0.04468789536065229],
 ['closest_bus_stop_mean_25', -0.04468789536065196],
 ['closest_bus_stop_mean_7', -0.0423769571255422],
 ['closest_bus_stop_sum_7', -0.042376957125542174],
 ['closest_bus_stop_mean_5', -0.040127934823078554],
 ['closest_bus_stop_sum_5', -0.040127934823078394],
 ['closest_bus_stop_mean_3', -0.037808251583550605],
 ['closest_bus_stop_sum_3', -0.037808251583550175],
 ['closest_bus_stop_mean_50', -0.037120456033797926],
 ['closest_bus_stop_sum_50', -0.037120456033797634],
 ['closest_bus_stop_sum_2', -0.03400879586258294],
 ['closest_bus_stop_mean_2', -0.03400879586258294],
 ['closest_bus_stop_sum_1', -0.0304164493706542],
 ['closest_bus_stop_mean_1', -0.0304164493706542],
 ['closest_bus_stop_sum_100', -0.02

In [708]:
bus_relevant_cols = bus_sorted_relevant_dist_cols[1::2]
bus_relevant_cols = [r[0] for r in bus_relevant_cols]

In [709]:
bus_relevant_cols = bus_relevant_cols[:5]

In [710]:
from grunnkrets import make_grunnkrets_df

full_population_dataframes = {}
full_pop_columns = []

for df_name, df in tqdm(store_dataframes.items()):
    full_population_dataframes[df_name] = make_grunnkrets_df(df)
    full_pop_columns = full_population_dataframes[df_name].columns

 25%|██▌       | 1/4 [00:05<00:17,  5.83s/it]/Users/oskarjor/NTNU/2022FALL/TDT4173/TDT4173/grunnkrets.py:101: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_population_df[f'{level}.income_density_log']  = np.log1p(full_population_df[f'{level}.income_density'])
/Users/oskarjor/NTNU/2022FALL/TDT4173/TDT4173/grunnkrets.py:100: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_population_df[f'{level}.income_density']  = full_population_df[f'{level}.total_income']/full_population_df[f'{level}.area_km2']
/Users/oskarjor/NTNU/2022

In [711]:
full_pop_columns = list(full_pop_columns[-184:])

In [712]:
for df_name, df in store_dataframes.items():
    store_dataframes[df_name] = df.merge(
        full_population_dataframes[df_name], 
        left_index=True,
        right_index=True,
        how="outer", 
        suffixes=('', '_redundant')
    )
    store_dataframes[df_name].drop(store_dataframes[df_name].filter(regex='_redundant$').columns, axis=1, inplace=True)

In [713]:
full_pop_dict = store_dataframes["train"][['log_revenue'] + full_pop_columns].corr().iloc[0].to_dict()
full_pop_sorted_relevant_dist_cols = [[k, v] for k, v in sorted(full_pop_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
full_pop_sorted_relevant_dist_cols

[['log_revenue', 1.0],
 ['grunnkrets_id.income_density', -0.0531280910350981],
 ['grunnkrets_id.pop_density', -0.05187407128809942],
 ['delomrade.mean_age', -0.03287734944784681],
 ['grunnkrets_id.income_density_log', -0.031740339253666185],
 ['grunnkrets_id.pop_density_log', -0.03126887632706533],
 ['grunnkrets_id.tot_pop_log', 0.030210578699937388],
 ['grunnkrets_id.total_income_log', 0.029350969510877567],
 ['kommune.c_age_19-30_ratio', 0.02932904639396046],
 ['delomrade.c_age_56-90_ratio', -0.027420865485196563],
 ['fylke.couple_with_children_income', -0.02661641304753561],
 ['fylke.couple_without_children_income', -0.024463870677286545],
 ['delomrade.income_density', -0.024121740457253914],
 ['delomrade.pop_density', -0.023520505417521008],
 ['kommune.mean_age', -0.023510329483582025],
 ['fylke.single_parent_children_6_to_17_years', -0.02334813517311376],
 ['fylke.single_parent_children_0_to_5_years', -0.022842423196133166],
 ['fylke.age_49-55', -0.02193552596251593],
 ['fylke.age

In [714]:
full_pop_relevant_cols = full_pop_sorted_relevant_dist_cols[1:8]
full_pop_relevant_cols = [r[0] for r in full_pop_relevant_cols]

In [715]:
full_pop_relevant_cols

['grunnkrets_id.income_density',
 'grunnkrets_id.pop_density',
 'delomrade.mean_age',
 'grunnkrets_id.income_density_log',
 'grunnkrets_id.pop_density_log',
 'grunnkrets_id.tot_pop_log',
 'grunnkrets_id.total_income_log']

In [716]:
fylke_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("fylke.")]
kommune_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("kommune.")]
delomrade_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("delomrade.")]
grunnkrets_relevant_features = [col_name for col_name in store_dataframes["train"].columns if col_name.startswith("grunnkrets_id.")]

In [717]:
from num_stores import add_num_stores_info

for df_name, df in store_dataframes.items():
    store_dataframes[df_name] = add_num_stores_info(df)

/Users/oskarjor/NTNU/2022FALL/TDT4173/TDT4173/num_stores.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped = geo_df.groupby([ level , cat]).sum()['count'].to_frame()
/Users/oskarjor/NTNU/2022FALL/TDT4173/TDT4173/num_stores.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  grouped = geo_df.groupby([ level , cat]).sum()['count'].to_frame()
/Users/oskarjor/NTNU/2022FALL/TDT4173/TDT4173/num_stores.py:19: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns wh

In [718]:
num_store_cols = list(store_dataframes["train"].columns[-64:])

In [719]:
num_store_dict = store_dataframes["train"][['log_revenue'] + num_store_cols].corr().iloc[0].to_dict()
num_store_sorted_relevant_cols = [[k, v] for k, v in sorted(num_store_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
num_store_sorted_relevant_cols

[['log_revenue', 1.0],
 ['fylke.plaace_cat_1_per_capita', 0.33418999230870905],
 ['fylke.plaace_cat_1_per_tot_income', 0.3283143301659981],
 ['kommune.plaace_cat_1_per_capita', 0.2819542574446876],
 ['kommune.plaace_cat_1_per_tot_income', 0.2686301934073755],
 ['fylke.plaace_cat_1_count', 0.1971101019208334],
 ['fylke.plaace_cat_4_per_capita', 0.18094002358872008],
 ['fylke.plaace_cat_4_per_tot_income', 0.17923520689726613],
 ['fylke.plaace_cat_3_per_capita', 0.16551994345727977],
 ['fylke.plaace_cat_3_per_tot_income', 0.16168295901219673],
 ['grunnkrets_id.plaace_cat_1_count', 0.13238053815084294],
 ['delomrade.plaace_cat_1_count', 0.10753666169151109],
 ['fylke.plaace_cat_4_count', 0.08518403193337079],
 ['kommune.plaace_cat_1_per_km2', 0.08318848084709703],
 ['kommune.plaace_cat_3_per_capita', 0.07637260668278562],
 ['fylke.plaace_cat_3_count', 0.07611427935502625],
 ['kommune.plaace_cat_4_per_capita', 0.07426649074316063],
 ['grunnkrets_id.plaace_cat_1_per_capita', 0.07059583136170

In [54]:
num_store_relevant_cols = num_store_sorted_relevant_cols[1:15]
num_store_relevant_cols = [r[0] for r in num_store_relevant_cols]

## Adding mean revenue (plaace_cat, level)

In [720]:
from avg_revenue import add_avg_revenue

for df_name, df in store_dataframes.items():
    store_dataframes[df_name] = add_avg_revenue(df, total=submission_with_whole_train)

In [721]:
mean_rev_cols = list(store_dataframes["train"].columns)[-20:]

In [722]:
mean_rev_dict = store_dataframes["val"][['log_revenue'] + mean_rev_cols].corr().iloc[0].to_dict()
mean_rev_sorted_relevant_cols = [[k, v] for k, v in sorted(mean_rev_dict.items(), key=lambda item: abs(item[1]), reverse=True)]
mean_rev_sorted_relevant_cols

[['log_revenue', 1.0],
 ['country.plaace_cat_0_mean_revenue', nan],
 ['country.plaace_cat_4_mean_revenue', 0.5086965809357913],
 ['country.plaace_cat_3_mean_revenue', 0.5040702021325972],
 ['fylke.plaace_cat_4_mean_revenue', 0.4941024731809248],
 ['fylke.plaace_cat_3_mean_revenue', 0.48932867443846284],
 ['kommune.plaace_cat_4_mean_revenue', 0.43662698654746335],
 ['kommune.plaace_cat_3_mean_revenue', 0.4299921137350498],
 ['delomrade.plaace_cat_4_mean_revenue', 0.41435511368423755],
 ['delomrade.plaace_cat_3_mean_revenue', 0.4060134442324854],
 ['country.plaace_cat_2_mean_revenue', 0.38622822057447803],
 ['fylke.plaace_cat_2_mean_revenue', 0.37675915499954615],
 ['country.plaace_cat_1_mean_revenue', 0.31597479646986176],
 ['fylke.plaace_cat_1_mean_revenue', 0.3093508806098102],
 ['kommune.plaace_cat_2_mean_revenue', 0.29652636200805776],
 ['kommune.plaace_cat_1_mean_revenue', 0.24695499192384035],
 ['delomrade.plaace_cat_2_mean_revenue', 0.2389899956305923],
 ['delomrade.plaace_cat_1_

In [723]:
mean_rev_relevant_cols = mean_rev_sorted_relevant_cols[1:]
mean_rev_relevant_cols = [r[0] for r in mean_rev_relevant_cols]

In [724]:
mean_rev_relevant_log_cols = []
for col in mean_rev_relevant_cols:
    for df_name, df in store_dataframes.items():
        store_dataframes[df_name][col + "_log"] = store_dataframes[df_name][col].apply(lambda x: np.log1p(x))
    mean_rev_relevant_log_cols.append(col + "_log")

In [603]:
from new_plaace_index import create_index_csv, add_new_plaace_index

create_index_csv()

In [725]:
for df_name, df in store_dataframes.items():
    store_dataframes[df_name] = add_new_plaace_index(store_dataframes[df_name])

# Read data directly from CSV (option 2)

## Train, val split

In [756]:
if submission_with_whole_train:
    stores_train = pd.read_csv("temp_data/full_train_features_train.csv", index_col=0)
    stores_extra = pd.read_csv("temp_data/full_train_features_extra.csv", index_col=0)
    stores_test = pd.read_csv("temp_data/full_train_features_test.csv", index_col=0)

    old_store_dataframes = {
        "train": stores_train, 
        "extra": stores_extra, 
        "test": stores_test, 
        }
else:
    stores_train = pd.read_csv("temp_data/full_features_train.csv", index_col=0)
    stores_val = pd.read_csv("temp_data/full_features_val.csv", index_col=0)
    stores_extra = pd.read_csv("temp_data/full_features_extra.csv", index_col=0)
    stores_test = pd.read_csv("temp_data/full_features_test.csv", index_col=0)

    old_store_dataframes = {
        "train": stores_train, 
        "extra": stores_extra, 
        "test": stores_test, 
        "val": stores_val
        }

In [783]:
rel_cols = ["store_id", "bounded_chain_name", "plaace_cat_4", "mean_revenue_4", "log_mean_revenue_4"]
store_ids = [
"990857423-990974489-88185", 
"948879220-972145203-18800", 
"951136751-971668970-6795", 
"980237508-980246167-71346", 
"999307795-999654649-475462"
]
old_store_dataframes["train"][rel_cols][old_store_dataframes["train"]["store_id"].isin(store_ids)]

,store_id,bounded_chain_name,plaace_cat_4,mean_revenue_4,log_mean_revenue_4
0,990857423-990974489-88185,BAKER KLAUSEN DRAMMEN,1.2.2.0,5.448447,1.863839
1,948879220-972145203-18800,OTHER,3.4.2.0,2.971692,1.379192
2,951136751-971668970-6795,EXPERT,2.3.1.0,13.944457,2.704340
3,980237508-980246167-71346,OTHER,2.9.1.0,6.400961,2.001610
9,999307795-999654649-475462,LEATHERMAN HOLDING,2.4.6.0,4.652183,1.732042


In [784]:
store_dataframes["train"][rel_cols][store_dataframes["train"]["store_id"].isin(store_ids)]

,store_id,bounded_chain_name,plaace_cat_4,mean_revenue_4,log_mean_revenue_4
0,990857423-990974489-88185,BAKER KLAUSEN DRAMMEN,1.2.2.0,4.886853,1.575334
36,948879220-972145203-18800,OTHER,3.4.2.0,1.731372,0.887143
88,951136751-971668970-6795,EXPERT,2.3.1.0,12.868180,2.223180
218,980237508-980246167-71346,OTHER,2.9.1.0,5.683577,1.684506
1736,999307795-999654649-475462,LEATHERMAN HOLDING,2.4.6.0,4.370217,1.479221


In [791]:
temp_df = store_dataframes["train"][store_dataframes["train"].plaace_cat_4 == "3.4.2.0"]
temp2_df = store_dataframes["val"][store_dataframes["val"].plaace_cat_4 == "3.4.2.0"]

In [794]:
temp_df.revenue.sum() / temp_df.shape[0]

2.9716923076923076

In [739]:
with open("relevant_cols.txt", "r") as f:
    data = f.readlines()

comp_relevant_cols = data[0].strip().split(",")
bus_relevant_cols = data[1].strip().split(",")
num_store_relevant_cols = data[2].strip().split(",")
full_pop_relevant_cols = data[3].strip().split(",")

In [740]:
comp_relevant_cols = comp_relevant_cols[::2]

In [741]:
comp_relevant_cols += [
    'sum_dist_to_nearest_10_comp_plaace_2',
    'sum_dist_to_nearest_5_comp_plaace_2', 
    'sum_dist_to_nearest_10_comp_plaace_1',
    'sum_dist_to_nearest_5_comp_plaace_1', 
    'sum_dist_to_nearest_1_comp_plaace_1',
    'sum_dist_to_nearest_1_comp_plaace_2'
    ]

In [742]:
bus_relevant_cols = bus_relevant_cols[::2]

In [743]:
num_store_relevant_cols

['fylke.plaace_cat_1_count',
 'fylke.plaace_cat_4_count',
 'fylke.plaace_cat_3_count',
 'delomrade.plaace_cat_1_count',
 'kommune.plaace_cat_1_count',
 'grunnkrets_id.plaace_cat_1_count']

# Transforming the data

In [726]:
from RMSLE import rmsle
from pred_var_utils import reverse_log1p_transform_pred_var
from sklearn.model_selection import GridSearchCV

In [744]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder


OE_categorical_features = ["bounded_chain_name", "kommune", "delomrade", "is_grocery", "plaace_cat_2", "plaace_cat_3", "plaace_cat_4", "new_plaace"]
OE_categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant")),
        ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
    ]
)

OH_categorical_features = ["plaace_cat_1"] #["fylke", "plaace_cat_2"]
OH_categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant")),
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)


numerical_features = ["lat", "lon", 
#"mean_revenue_1", "mean_revenue_2", "mean_revenue_3", "mean_revenue_4", 
"log_mean_revenue_1", "log_mean_revenue_2", "log_mean_revenue_3", "log_mean_revenue_4", 
"fylke.plaace_cat_1_mean_revenue_log", #"fylke.plaace_cat_3_mean_revenue_log", 
"log_chain_mean_revenue"
] + full_pop_relevant_cols + comp_relevant_cols + bus_relevant_cols + num_store_relevant_cols #+ mean_rev_relevant_log_cols
numerical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")), 
        ("scaler", StandardScaler(with_mean=True, with_std=True))]
)


preprocessor = ColumnTransformer(
    transformers=[
        ("oe_cat", OE_categorical_transformer, OE_categorical_features),
        ("oh_cat", OH_categorical_transformer, OH_categorical_features),
        ("num", numerical_transformer, numerical_features),
    ],
    remainder='drop'
)

preprocessor = ColumnTransformer(
    transformers=[
        ("oe_cat", OE_categorical_transformer, OE_categorical_features),
        ("oh_cat", OH_categorical_transformer, OH_categorical_features),
        ("num", numerical_transformer, numerical_features),
    ],
    remainder='drop'
)


X_train = preprocessor.fit_transform(store_dataframes["train"])
if not submission_with_whole_train:
    X_val = preprocessor.transform(store_dataframes["val"])

In [745]:
y_train = np.array(store_dataframes["train"].log_revenue)
if not submission_with_whole_train:
    y_val = np.array(store_dataframes["val"].revenue)
mean_y = y_train.mean()
std_y = y_train.std()

y_train -= mean_y
y_train /= std_y

# PCA

In [ ]:
PCA_cols_dict = {
    "OE": [],
    "OH": [],  
    "num": []
}

reduntant_cols = ["store_id", "year", "store_name", "plaace_hierarchy_id", "sales_channel_name", "address", "revenue", "log_revenue", "point"]

for col_name, dtype in store_dataframes["train"].dtypes.to_dict().items():
    if(col_name in reduntant_cols):
        continue
    if(dtype == int or dtype == float):
        _type = "num"
    elif(dtype == bool):
        _type = "OE"
    elif(dtype == object):
        if(store_dataframes["train"][col_name].nunique() <= 10):
            _type = "OH"
        else:
            _type = "OE"
    else:
        print(f"Unknown type {dtype} encountered for columns {col_name}")
    PCA_cols_dict[_type].append(col_name)
    

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("oe_cat", OE_categorical_transformer, PCA_cols_dict["OE"]),
        ("oh_cat", OH_categorical_transformer, PCA_cols_dict["OH"]),
        ("num", numerical_transformer, PCA_cols_dict["num"]),
    ],
    remainder='drop'
)

In [ ]:
pre_PCA_X_train = preprocessor.fit_transform(store_dataframes["train"])
if not submission_with_whole_train:
    pre_PCA_X_val = preprocessor.transform(store_dataframes["val"])

In [ ]:
pca = PCA(n_components=25)
PCA_X_train = pca.fit_transform(pre_PCA_X_train)
if not submission_with_whole_train:
    PCA_X_val = pca.transform(pre_PCA_X_val)

In [ ]:
print(pca.explained_variance_ratio_)

print(pca.singular_values_)

# Training the model

## Random Forest

In [496]:
from sklearn.ensemble import RandomForestRegressor

# best params (8.11)
# {'max_features': 8, 'min_samples_leaf': 4, 'min_samples_split': 32, 'n_estimators': 500}
# rmsle(on val): 0.719099511243053

rf = RandomForestRegressor(random_state=0, n_jobs=-1, max_features=8, min_samples_leaf=4, min_samples_split=32, n_estimators=500)
rf_params = {
    "n_estimators" : (100, 250, 500), 
    "max_features" : (2, 4, 8), 
    "min_samples_split" : (4, 8, 16, 32), 
    "min_samples_leaf" : (1, 2, 4), 
    }

rf_clf = GridSearchCV(rf, rf_params, verbose=2)

In [497]:
rf.fit(X_train, y_train)
rf_y_pred = reverse_log1p_transform_pred_var(rf.predict(X_val), std_y=std_y, mean_y=mean_y)
rmsle(y_val, rf_y_pred)

0.7295688027831453

In [221]:
#rf_clf.best_params_

In [222]:
#rf_clf.best_params_

In [223]:
rf_y_pred = reverse_log1p_transform_pred_var(rf.predict(X_val), std_y=std_y, mean_y=mean_y)


In [224]:
rmsle(y_val, rf_y_pred)

0.7196111316627671

## Linear Regression

In [479]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression(n_jobs=-1)

In [480]:
lr.fit(X_train, y_train)

LinearRegression(n_jobs=-1)

In [481]:
lr_y_pred = reverse_log1p_transform_pred_var(lr.predict(X_val), std_y, mean_y)
lr_y_pred = np.array([max(0, xi) for xi in lr_y_pred])

In [482]:
rmsle(y_val, lr_y_pred)

0.7485071979848835

## Light GBM

In [746]:
from lightgbm import LGBMRegressor

# best params (8.11)
# {'learning_rate': 0.05, 'min_child_samples': 16, 'min_split_gain': 0, 'n_estimators': 100, 'num_leaves': 25}
# rmsle(on val): 0.7208999886795342

lgbm = LGBMRegressor(
    random_state=0, 
    n_jobs=-1, 
    learning_rate=0.05, 
    min_child_samples=16, 
    min_split_gain=0, 
    n_estimators=100, 
    num_leaves=25
    )

lgbm_params = {
    "num_leaves" : (10, 25, 31, 75), 
    "learning_rate" : (0.05, 0.1, 0.25), 
    "n_estimators" : (50, 100, 250), 
    "min_split_gain" : (0, 0.01), 
    "min_child_samples" : (4, 8, 16, 32), 
    #"reg_alpha" : (0, 0.01, 0.1), 
    #"reg_lambda" : (0, 0.01, 0.1), 
    }

lgbm_clf = GridSearchCV(lgbm, lgbm_params, verbose=2)

In [747]:
lgbm.fit(X_train, y_train)

LGBMRegressor(learning_rate=0.05, min_child_samples=16, min_split_gain=0,
              num_leaves=25, random_state=0)

In [748]:
#lgbm_clf.best_params_

In [749]:
lgbm_y_pred = reverse_log1p_transform_pred_var(lgbm.predict(X_val), std_y, mean_y)
lgbm_y_pred = np.array([max(0, xi) for xi in lgbm_y_pred])

In [750]:
rmsle(y_val, lgbm_y_pred)

0.7344922753073084

## CatBoost

In [751]:
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV


# best params (8.11)
# {'depth': 6, 'l2_leaf_reg': 5, 'learning_rate': 0.05, 'eval_metric': 'RMSE'}
# rmsle(on val) = 0.7190096805219454

cb = CatBoostRegressor(
    random_seed=0, 
    verbose=False, 
    eval_metric="RMSE", 
    depth=6, 
    l2_leaf_reg= 5, 
    learning_rate= 0.05
    )

cb_params = grid = {
    'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.3],
    'depth': [5, 6, 8, 10],
    'l2_leaf_reg': [4, 5, 6, 7, 8], 
    }

# cb_clf = cb.grid_search(cb_params, X=X_train, y=y_train)

In [752]:
#cb_clf["params"]

In [753]:
cb.fit(X_train, y_train)

In [754]:
cb_y_pred = reverse_log1p_transform_pred_var(cb.predict(X_val), std_y, mean_y)
cb_y_pred = np.array([max(0, xi) for xi in cb_y_pred])

In [755]:
rmsle(y_val, cb_y_pred)

0.7366041432934709

## Stacking classifiers

In [ ]:
from sklearn.ensemble import StackingRegressor

estimators = [
    ('rf', rf), 
    ('cb', cb), 
    ('lgbm', lgbm), 
]

rf_end_params = {
    "n_estimators" : (50, 100, 250), 
    "max_features" : (1, 2, 3), 
    "min_samples_split" : (16, 32), 
    "min_samples_leaf" : (2, 4, 8), 
    }

rf_end = RandomForestRegressor(random_state=0, n_jobs=-1, n_estimators=50, max_features=3)

reg = StackingRegressor(
    estimators=estimators,
    final_estimator=rf_end
    )

In [ ]:
reg.fit(X_train, y_train)

In [ ]:
reg_y_pred = reverse_log1p_transform_pred_var(reg.predict(X_val), std_y, mean_y)

In [ ]:
rmsle(y_pred=reg_y_pred, y_true=y_val)

## Creating the submission

In [ ]:
# Predict on the test set 
X_test = preprocessor.transform(store_dataframes["test"])
y_test_pred = reverse_log1p_transform_pred_var(cb.predict(X_test), std_y, mean_y)

# Generate submission dataframe 
# NOTE: It is important that the ID and predicted values match
submission = pd.DataFrame()
submission['id'] = stores_test.store_id 
submission['predicted'] = np.asarray(y_test_pred)

# Save it to disk (`index=False` means don't save the index in the csv)
submission.to_csv('submission.csv', index=False)

# Create CSV of dataframes

In [144]:
for df_name, df in store_dataframes.items():
    filepath = Path(f"temp_data/full_features_{df_name}_12_nov.csv")  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    df.to_csv(filepath, index=True)